In [1]:
import os
import re
import time
import random
import asyncio
import warnings
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor,  as_completed
import pandas as pd
import nest_asyncio
import cloudscraper
from htmldate import find_date
from bs4 import BeautifulSoup
from newspaper import Article
import ipywidgets as widgets
from IPython.display import display

warnings.filterwarnings('ignore')

In [2]:
companies = {
    "Tesla": "tesla-motors",
    "NVIDIA": "nvidia-corp",
    "Apple": "apple-computer-inc",
    "Microsoft": "microsoft-corp",
    "Amazon": "amazon-com-inc",
    "Google": "google-inc",
    "Meta": "facebook-inc",
    "Netflix": "netflix,-inc.",
    "AMD": "advanced-micro-devices",
}

# Create a dropdown widget
company_dropdown = widgets.Dropdown(
    options=list(companies.keys()),
    value='Tesla',
    description='Company:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

display(company_dropdown)

Dropdown(description='Company:', layout=Layout(width='300px'), options=('Tesla', 'NVIDIA', 'Apple', 'Microsoft…

In [3]:
COMPANY = companies[company_dropdown.value]
MAX_PAGE    = 2
MAX_WORKERS = 50              
MAX_RETRIES = 8              

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Referer": "https://www.investing.com/",
    "DNT": "1"
}

scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'darwin',
        'mobile': False
    },
    delay=2
)

def fetch_page(page: int):
    global ticker
    #Equity
    url = f"https://www.investing.com/equities/{COMPANY}-news/{page}"
    #Index
    #url = f"https://www.investing.com/indices/nq-100-news/{page}"
    #url = f"https://www.investing.com/indices/us-spx-500-news/{page}"
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = scraper.get(url, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "lxml")
            h1_tag = soup.find('h1', class_='mb-2.5') # Use a specific class or combination of classes for robustness
            full_text = h1_tag.text.strip()
            match = re.search(r'\(([^)]+)\)', full_text)
            ticker = match.group(1)
                    
            anchors = soup.select(
                'ul[data-test="news-list"] '
                'li article a[data-test="article-title-link"]'
            )
            return [a["href"] for a in anchors if a.has_attr("href")]
        except Exception as e:
            if attempt < MAX_RETRIES:
                backoff = 2 ** (attempt - 1) + random.random()
                time.sleep(backoff)
            else:
                print(f"Page {page} failed after {MAX_RETRIES}: {e}")
    return []

def robust_scrape():
    first = fetch_page(1)
    PER_PAGE = len(first)
    if PER_PAGE == 0:
        raise RuntimeError("Failed to fetch the first page. Please check headers or cookies and try again.")
    print(f"Detected {PER_PAGE} links per page, expecting {PER_PAGE * MAX_PAGE} total")

    results = {1: first}
    pages = list(range(2, MAX_PAGE + 1))

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
        futures = {pool.submit(fetch_page, p): p for p in pages}
        for fut in as_completed(futures):
            p = futures[fut]
            results[p] = fut.result()

        for round in range(1, MAX_RETRIES + 1):
            bad = [p for p, links in results.items() if len(links) != PER_PAGE]
            if not bad:
                print(f"All pages OK after {round-1} retries")
                break
            print(f"Retry round {round} for pages: {bad}")
            futures = {pool.submit(fetch_page, p): p for p in bad}
            for fut in as_completed(futures):
                p = futures[fut]
                results[p] = fut.result()
        else:
            print("Retry limit reached; some pages may still be incomplete.")

    total_fetched = sum(len(links) for links in results.values())
    expected = PER_PAGE * MAX_PAGE
    print(f"Total links fetched (including duplicates): {total_fetched} (expected {expected})")

    all_links = set(link for links in results.values() for link in links)
    print(f"Final: got {len(all_links)} unique URLs (expected {expected})")
    return list(all_links)

if __name__ == "__main__":
    links = robust_scrape()

Detected 10 links per page, expecting 20 total
All pages OK after 0 retries
Total links fetched (including duplicates): 20 (expected 20)
Final: got 20 unique URLs (expected 20)


In [4]:
nest_asyncio.apply()

FETCH_WORKERS = min(32, os.cpu_count() * 4)  
PROCESS_WORKERS = os.cpu_count() or 4
MAX_FETCH_RETRIES = 5                      
RETRY_DELAY = 1                             
TICKER = ticker
scraper = cloudscraper.create_scraper()

def is_placeholder(html: str) -> bool:
    lower = html.lower() if html else ""
    return (
        'temporarily down for maintenance' in lower
        or 'just a moment' in lower
        or "we're temporarily down" in lower
    )

def safe_find_datetime(url, html_content=None):
    try:
        # Strategy 1: Use htmldate library to extract date from URL
        dt = find_date(url)
        if dt:
            return dt, "00:00"  # Return with default time if date found
    except:
        pass
    
    if html_content:
        # Strategy 2: Look for American format with AM/PM
        m = re.search(r"(\d{1,2}/\d{1,2}/\d{4}),\s*(\d{1,2}:\d{2}\s*(?:AM|PM))", html_content)
        if m:
            ds, ts = m.groups()
            try:
                dt = datetime.strptime(f"{ds}, {ts}", "%m/%d/%Y, %I:%M %p")
                return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M")
            except:
                pass
        
        # Strategy 3: Look for numeric dates with 24-hour time format
        m = re.search(r"(\d{2}/\d{2}/\d{4}),\s*(\d{2}:\d{2})", html_content)
        if m:
            ds, ts = m.groups()
            # Try both European and American date formats
            for fmt in ("%d/%m/%Y, %H:%M", "%m/%d/%Y, %H:%M"):
                try:
                    dt = datetime.strptime(f"{ds}, {ts}", fmt)
                    return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M")
                except:
                    continue
    
    now = datetime.now()
    return now.strftime("%Y-%m-%d"), now.strftime("%H:%M")

def fetch_html(url, idx, total):
    for attempt in range(1, MAX_FETCH_RETRIES + 1):
        try:
            resp = scraper.get(url, timeout=30)
            html = resp.text
            if is_placeholder(html):
                raise RuntimeError('Placeholder')
                
            print(f"[Fetch][{idx}/{total}][ok]")
            return url, html
            
        except Exception:
            print(f"[Fetch][{idx}/{total}][retry {attempt}]")
            if attempt < MAX_FETCH_RETRIES:
                time.sleep(RETRY_DELAY)
                
    print(f"[Fetch error] {idx}/{total}: failed after {MAX_FETCH_RETRIES} retries")
    return url, None

def process_article(arg):
    url, html = arg
    if not html:
        return None
        
    art = Article(url)
    art.set_html(html)
    
    try:
        art.parse()
    except:
        return None
        
    text = art.text or ""
    title = (art.title or "").strip() or "No title"
    
    date, tm = safe_find_datetime(url, html)
    
    # Return combined data using dictionary unpacking
    return {'ticker': TICKER, 'publish_date': date, 'publish_time': tm,
             'title': title, 'body_text': text, 'url': url}

async def scrape_all(urls):
    total = len(urls)
    loop = asyncio.get_event_loop()
    
    # Phase 1: Fetch HTML content from all URLs in parallel
    with ThreadPoolExecutor(max_workers=FETCH_WORKERS) as fetch_pool:
        # Create fetch tasks and run them through the thread pool
        fetch_tasks = [loop.run_in_executor(fetch_pool, fetch_html, u, i+1, total)
                       for i, u in enumerate(urls)]
        # Wait for all fetch tasks to complete
        fetched = await asyncio.gather(*fetch_tasks)

    # Phase 2: Process all fetched HTML content in parallel
    records = []
    with ThreadPoolExecutor(max_workers=PROCESS_WORKERS) as proc_pool:
        # Submit processing tasks only for URLs with successful fetches
        futures = {
            proc_pool.submit(process_article, fr): fr[0]
            for fr in fetched if fr[1]  # Skip URLs where HTML is None
        }
        
        # Process results as they complete
        for i, fut in enumerate(as_completed(futures), 1):
            res = fut.result()
            print(f"[Process][{i}/{total}] {futures[fut]}")
            if res:
                records.append(res)
                
    # Convert results to DataFrame
    return pd.DataFrame(records)

# ——— Main entry point function ———
def main(links):
    df = asyncio.get_event_loop().run_until_complete(scrape_all(links))
    return df

# Execute the main function if this script is run directly
if __name__ == '__main__':
    df = main(links)

[Fetch][5/20][ok]
[Fetch][4/20][ok]
[Fetch][17/20][ok]
[Fetch][1/20][ok]
[Fetch][16/20][ok]
[Fetch][18/20][ok]
[Fetch][12/20][ok]
[Fetch][10/20][ok]
[Fetch][2/20][ok]
[Fetch][8/20][ok]
[Fetch][9/20][ok]
[Fetch][6/20][ok]
[Fetch][15/20][ok]
[Fetch][20/20][ok]
[Fetch][13/20][ok]
[Fetch][19/20][ok]
[Fetch][11/20][ok]
[Fetch][7/20][ok]
[Fetch][14/20][ok]
[Fetch][3/20][ok]
[Process][1/20] https://www.investing.com/news/pro/td-cowen-maintains-tesla-at-buy-with-a-price-target-of-33000-4106202
[Process][2/20] https://www.investing.com/news/stock-market-news/tesla-seeks-confidentiality-on-robotaxi-safety-answers-to-nhtsa-93CH-4106440
[Process][3/20] https://www.investing.com/news/stock-market-news/nhtsa-contacts-tesla-on-robotaxi-issues-seen-in-online-videos-bloomberg-news-reports-4106795
[Process][4/20] https://www.investing.com/news/stock-market-news/europeans-seek-digital-sovereignty-as-us-tech-firms-embrace-trump-4106318
[Process][5/20] https://www.investing.com/news/analyst-ratings/tesla-p

In [5]:
df=df.sort_values(by=['publish_date', 'publish_time'], ascending=[False,False]).reset_index(drop=True)
pd.set_option('display.max_columns', None)

In [6]:
empty_body_count = df[df['body_text'] == ''].shape[0]
print(f"Number of articles with empty body_text: {empty_body_count}")

Number of articles with empty body_text: 0


In [7]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv("../SentimentAnalysis/GPT/secret.env")

api_key = os.getenv("GEMINI_API_KEY")
mongo_connection_string = os.getenv("MONGO_CONNECTION_STRING")

if api_key is None:
    print("Error: GEMINI_API_KEY not found in .env file or environment variables.")
else:
    print("GEMINI_API_KEY loaded successfully.")

try:
    client = MongoClient(mongo_connection_string)
    db = client['stock_news_db']
    collection = db['news_data']

    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")

except Exception as e:
    print(f"An error occurred: {e}")

GEMINI_API_KEY loaded successfully.
Pinged your deployment. You successfully connected to MongoDB!


In [8]:
import google.generativeai as genai
from tqdm import tqdm

genai.configure(api_key=api_key)
generation_config = genai.GenerationConfig(
        temperature=0,
)
model = genai.GenerativeModel("gemini-2.5-flash-preview-04-17", generation_config=generation_config)

In [9]:
from google.api_core import retry

def is_retryable(e) -> bool:
    if retry.if_transient_error(e):
        return True
    elif (isinstance(e, genai.errors.ClientError) and e.code == 429):
        return True
    elif (isinstance(e, genai.errors.ServerError) and e.code == 503):
        return True
    else:
        return False

@retry.Retry(predicate=is_retryable)
def generate_content_with_rate_limit(prompt):
  return model.generate_content(prompt).text

In [10]:
prompt = """You are a financial news analyst specializing in stock market impact. Your task is to analyze the provided news article, summarize its core content concisely, determine its sentiment (positive, negative, or neutral), and assess its importance to the specified stock.

Here is the news from stock [STOCK] title and body:
---
[TITLE]
---
[BODY]
---

Please provide your analysis in the following format (Don't forget to make space between the sections as shown):

**Sentiment:**
[Positive / Negative / Neutral]

**Summary:**
[Your concise summary of the article, typically 2-3 sentences.]

**Reasoning for Sentiment:**
[Brief explanation (1-2 sentences) of why you categorized the sentiment as such, referencing key points or tone from the article.]

**Importance to Stock [STOCK]:**
[1-5, where 1 is minimal importance and 5 is very high importance.Answer in 1-5 only, no explanation.] (Answer only in number 1-5)

**Reasoning for Importance:**
[Brief explanation (1-2 sentences) of why you assigned this importance score, referencing specific details from the article that would impact the stock.]"""

In [11]:
predicted = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 
    current_stock = row.get("ticker", "news")

    filled_prompt = prompt.replace("[STOCK]", current_stock)
    filled_prompt = filled_prompt.replace("[TITLE]", row["title"])
    filled_prompt = filled_prompt.replace("[BODY]", row["body_text"])

    try:
        response = generate_content_with_rate_limit(filled_prompt)
        finalprediction = response.strip()
        if not finalprediction:
            print(f"Row {index}: LLM returned an empty string.")
            predicted.append("LLM_EMPTY_RESPONSE")
        else:
            predicted.append(finalprediction)
    except ValueError as ve:
        print(f"Row {index}: ValueError - {ve}. Appending 'ERROR_VALUE_ERROR'.")
        predicted.append("ERROR_VALUE_ERROR")
        continue
    except Exception as e:
        if "429 Too Many Requests" in str(e) or "quota" in str(e).lower():
            print(f"Row {index}: Rate Limit Exceeded or Quota Error - {e}. Appending 'ERROR_RATE_LIMIT'.")
            predicted.append("ERROR_RATE_LIMIT")
        elif "safety" in str(e).lower() or "blocked" in str(e).lower():
             print(f"Row {index}: Content Safety/Blocked - {e}. Appending 'ERROR_SAFETY_BLOCKED'.")
             predicted.append("ERROR_SAFETY_BLOCKED")
        else:
            print(f"Row {index}: Unexpected Error - {e}. Appending 'ERROR_UNEXPECTED'.")
            predicted.append("ERROR_UNEXPECTED")
        continue

Prompting: 100%|██████████| 20/20 [01:41<00:00,  5.07s/it]


In [12]:
import numpy as np
predicted = np.array(predicted)
df["predicted"] = predicted

In [13]:
df["sentiment"] = df["predicted"].apply(lambda x: x.split("\n")[1].strip() if len(x.split("\n")) > 1 else None)
df["importance"] = df["predicted"].apply(lambda x: x.split("\n")[10].strip() if len(x.split("\n")) > 10 else None)
df["summary"] = df["predicted"].apply(lambda x: x.split("\n")[4].strip() if len(x.split("\n")) > 4 else None)

In [14]:
df = df[df['sentiment'].isin(['Positive', 'Negative', 'Neutral'])]
df = df[df['importance'].isin(['1', '2', '3', '4', '5'])]

In [15]:
model = genai.GenerativeModel("gemini-2.0-flash-001", generation_config=generation_config)
prompt = """Translate the following English sentence to Thai. Do not translate proper nouns, company names, product names, abbreviations, or technical terms — keep them in English. Do not provide any explanation, just the translation.
[TEXT]"""

In [16]:
translate = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 

    filled_prompt = prompt.replace("[TEXT]", row["summary"])

    try:
        response = generate_content_with_rate_limit(filled_prompt)
        finalprediction = response.strip()
        if not finalprediction:
            print(f"Row {index}: LLM returned an empty string.")
            translate.append("LLM_EMPTY_RESPONSE")
        else:
            translate.append(finalprediction)
    except ValueError as ve:
        print(f"Row {index}: ValueError - {ve}. Appending 'ERROR_VALUE_ERROR'.")
        translate.append("ERROR_VALUE_ERROR")
        continue
    except Exception as e:
        if "429 Too Many Requests" in str(e) or "quota" in str(e).lower():
            print(f"Row {index}: Rate Limit Exceeded or Quota Error - {e}. Appending 'ERROR_RATE_LIMIT'.")
            translate.append("ERROR_RATE_LIMIT")
        elif "safety" in str(e).lower() or "blocked" in str(e).lower():
             print(f"Row {index}: Content Safety/Blocked - {e}. Appending 'ERROR_SAFETY_BLOCKED'.")
             translate.append("ERROR_SAFETY_BLOCKED")
        else:
            print(f"Row {index}: Unexpected Error - {e}. Appending 'ERROR_UNEXPECTED'.")
            translate.append("ERROR_UNEXPECTED")
        continue

Prompting:   0%|          | 0/20 [00:00<?, ?it/s]

Prompting: 100%|██████████| 20/20 [00:24<00:00,  1.22s/it]


In [17]:
TICKER = current_stock

In [18]:
if 'predicted' in df.columns:
    df.drop(columns=['predicted'], inplace=True)
if 'body_text' in df.columns:
    df.drop(columns=['body_text'], inplace=True)
now = datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M").strip().replace(' ', '_')
filename = f"Gemini_{TICKER}_{date_time}.csv"

In [19]:
df["translate"] = translate
df.to_csv(f"Data/{filename}", index=False)

In [20]:
complete_dict=df.to_dict(orient='records')

result = collection.insert_many(complete_dict,ordered=True)
print(f"Successfully inserted document with id: {result.inserted_ids}")

Successfully inserted document with id: [ObjectId('685a277022b8db3ec7f044e4'), ObjectId('685a277022b8db3ec7f044e5'), ObjectId('685a277022b8db3ec7f044e6'), ObjectId('685a277022b8db3ec7f044e7'), ObjectId('685a277022b8db3ec7f044e8'), ObjectId('685a277022b8db3ec7f044e9'), ObjectId('685a277022b8db3ec7f044ea'), ObjectId('685a277022b8db3ec7f044eb'), ObjectId('685a277022b8db3ec7f044ec'), ObjectId('685a277022b8db3ec7f044ed'), ObjectId('685a277022b8db3ec7f044ee'), ObjectId('685a277022b8db3ec7f044ef'), ObjectId('685a277022b8db3ec7f044f0'), ObjectId('685a277022b8db3ec7f044f1'), ObjectId('685a277022b8db3ec7f044f2'), ObjectId('685a277022b8db3ec7f044f3'), ObjectId('685a277022b8db3ec7f044f4'), ObjectId('685a277022b8db3ec7f044f5'), ObjectId('685a277022b8db3ec7f044f6'), ObjectId('685a277022b8db3ec7f044f7')]


In [21]:
data_filepath_for_streamlit = os.path.join("..", "CompletePipeline", "Data", filename)

print(f"Path to be passed to Streamlit: {data_filepath_for_streamlit}")

!streamlit run ../Visualization/stock_app.py "{data_filepath_for_streamlit}"

Path to be passed to Streamlit: ../CompletePipeline/Data/Gemini_TSLA_2025-06-24_11-20.csv

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.16.0.2:8501

YF.download() has changed argument auto_adjust default to True
^C
  Stopping...


In [107]:
df.head()

,ticker,publish_date,publish_time,title,url,sentiment,importance,summary,translate
0,META,2025-06-09,10:02,Edgewater comments on Meta Platforms By Invest...,https://www.investing.com/news/pro/edgewater-c...,Neutral,1,The provided text consists entirely of standar...,ข้อความที่ให้มาประกอบด้วยข้อความเปิดเผยความเสี...
1,META,2025-06-09,06:06,"Apple faces AI, regulatory challenges as it wo...",https://www.investing.com/news/stock-market-ne...,Negative,5,The article details the significant technical ...,บทความนี้ให้รายละเอียดเกี่ยวกับความท้าทายทางเท...
2,META,2025-06-09,05:24,Meta Platforms stock target reiterated at $750...,https://www.investing.com/news/analyst-ratings...,Positive,4,Citizens JMP analysts reiterated their Market ...,นักวิเคราะห์ของ Citizens JMP ย้ำคำแนะนำ Market...
3,META,2025-06-09,04:39,Facebook Inc receives Investment Bank Analyst ...,https://www.investing.com/news/pro/citizens-jm...,Neutral,1,The article announces that Facebook Inc (META)...,บทความประกาศว่า Facebook Inc (META) ได้รับการป...
4,META,2025-06-08,19:59,South Korean conservatives looking for rebirth...,https://www.investing.com/news/world-news/sout...,Negative,1,South Korea's conservative People Power Party ...,พรรคพลังประชาชน (People Power Party) ฝ่ายอนุรั...
